This notebook was created by Hubert Koh for the purpose of IBM Capstone Week 3-1 
Part 1: Scrape information from wikipage and clean it up  



## Part 1

In [13]:
# load libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np



In [14]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(website_url,'lxml')
#using beautifulsoup to scrap url 

In [15]:
#find class table from html and lopp it into a table 
My_tablesoup = soup.find('table',{'class':"wikitable sortable"})
My_table = []
for rows in My_tablesoup.find_all('td'):
    row = rows.text
    row = row.replace('\n', '')
    My_table.append(row)


In [16]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
df=[]
df=pd.DataFrame(columns=column_names)

In [17]:
#transform the downloaded data into a table  
df.iloc[:,0]=My_table[::3]
df.iloc[:,1]=My_table[1::3]
df.iloc[:,2]=My_table[2::3]
df.replace("Not assigned", np.nan, inplace = True)
df.dropna(subset=["Borough"], axis=0, inplace = True)
df.reset_index(drop=True, inplace=True)

for i in range(0, df.shape[0]):
    if pd.isnull(df.loc[i,'Neighborhood']):
        df.replace(df.loc[i,'Neighborhood'], df.loc[i,'Borough'],inplace=True)
df.head(15)



,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [18]:
df['Neighborhood'] = df[['PostalCode','Borough','Neighborhood']].groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x)) 
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

df.head(12)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [19]:
df.shape


(103, 3)